In [1]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


0.40


In [3]:
onto = get_ontology("./WIP.owl").load()

In [4]:
SH = [x.label[0] for x in onto.Stakeholder.instances() if len(x.label) >= 1]
len(SH)

16001

In [5]:
SHs = [x[:-1] for x in SH if x.endswith("s")]
SHs = [x+"s" for x in SHs if x in SH]
SH = [x for x in SH if x not in SHs]
len(SHs),len(SH)

(546, 15455)

# Classification

In [6]:
import pandas as pd
import json
import glob, os
import requests
from datetime import datetime
import hashlib

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv('.env')

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
CONTEXT = """You are given a list of stakeholders. Classify them using 10 groups of stakeholders you define. Answer only as a bullet point list structure, similar to:
## Name of stakeholders group
- stakeholder A
- stakeholder C 
## Name of stakeholders group
- stakeholder B
etc..
"""
NChunks = 1000

NB = len(SH)//NChunks
CATS = []
for k in range(NB):
    QUESTION =  "The stakeholders are:\n\n* "+"\n* ".join(SH[(k*NChunks):(k+1)*NChunks]) 
    REQ = {
        "context": CONTEXT,
        "question": QUESTION,
        "token": TOK3N,
        "overwrite": True,
        "source": "local-evalrisksT3.5"
    }

    x = requests.post(URL+"ask/", json = REQ)
    CATS.append(json.loads(x.text)["answer"])

In [10]:
titles = []
for k in range(len(CATS)):
    [titles.append(x.replace("#","").strip()) for x in CATS[k].split("\n") if x.startswith("#")]
    titles.sort()
pd.DataFrame(titles).to_excel("../../support/classif_auto_stakeholders.xlsx")

In [11]:
titles

['Building Occupants',
 'Building and construction stakeholders',
 'Business and Economic',
 'Business and Workplace',
 'Business and corporate stakeholders',
 'Business and economic stakeholders',
 'Clinical research stakeholders',
 'Commercial and Workplace Stakeholders',
 'Commercial and business stakeholders',
 'Communication and marketing stakeholders',
 'Communities and Residents',
 'Community Engagement and Awareness',
 'Community and Residential Stakeholders',
 'Community and social equity stakeholders',
 'Community and social stakeholders',
 'Community and urban planning stakeholders',
 'Community stakeholders',
 'Data Analysis and Research',
 'Data and Technology',
 'Data governance stakeholders',
 'Decision Makers and Legal',
 'Economic and Labor Sectors',
 'Economic and business entities',
 'Economic stakeholders',
 'Economic stakeholders',
 'Education',
 'Education and Research Stakeholders',
 'Education and public information',
 'Education and research stakeholders',
 'Ed

In [13]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

In [27]:
QUESTION =  "The risks are:\n\n* "+"\n* ".join(titles) 

#answer = ask("Out the following list, classify the risks  \n\n",Rs,prefix="risk_classification",functions=classifFct,overwrite=True)
prefix ="ff"
overwrite = False
REQ = {
    "context": "give me a bullet point list of 50 stakeholders groups, which could capture the following list of stakeholders. Don't use 'stakeholder' in their names.",
    "question": QUESTION,
    "token": TOK3N,
    "overwrite": overwrite,
    "source": "local-findsthT3.5"
}

H = h.hashme(REQ["context"]+REQ["question"])
cached = "cache/"+prefix+"_"+H+".json"
if not os.path.isfile(cached) or overwrite:
    x = requests.post(URL+"ask/", json = REQ)
    answer = json.loads(x.text)["answer"]
    h.svt(cached,answer)
else:
    answer = h.ldt(cached)

In [29]:
ls = [x.strip() for x in answer.split("- ") if len(x)]
ls.sort()
ls

['Building and construction industry',
 'Building occupants',
 'Business and economic sector',
 'Clinical research community',
 'Commercial workplace sector',
 'Community engagement organizations',
 'Corporate entities',
 'Data analysts and researchers',
 'Education institutions',
 'Environmental conservation groups',
 'Financial and investment groups',
 'General public',
 'Global trade and supply chain stakeholders',
 'Governance and policy entities',
 'Government and public health authorities',
 'Health and medical professionals',
 'Healthcare institutions',
 'Housing and real estate industry',
 'Immigration and migrant population representatives',
 'Individuals and communities',
 'Infrastructure stakeholders',
 'Labor and economic sectors',
 'Legal decision makers',
 'Local communities and residents',
 'Logistics and supply chain sector',
 'Marketing and communication professionals',
 'Media and communication networks',
 'Mental health and social well-being supporters',
 'Mobility a

In [30]:
LS = ['Building and construction industry',
 'Building occupants',
 'Business and economic sector',
 'Clinical research community',
 'Commercial workplace sector',
 'Community engagement organizations',
 'Corporate entities',
 'Data analysts and researchers',
 'Education institutions',
 'Environmental conservation groups',
 'Financial and investment groups',
 'General public',
 'Global trade and supply chain stakeholders',
 'Governance and policy entities',
 'Government and public health authorities',
 'Health and medical professionals',
 'Healthcare institutions',
 'Housing and real estate industry',
 'Immigration and migrant population representatives',
 'Individuals and communities',
 'Infrastructure stakeholders',
 'Labor and economic sectors',
 'Legal decision makers',
 'Local communities and residents',
 'Logistics and supply chain sector',
 'Marketing and communication professionals',
 'Media and communication networks',
 'Mental health and social well-being supporters',
 'Mobility and transportation organizations',
 'Occupational and work-related organizations',
 'Public health and safety bodies',
 'Public policy makers',
 'Research and academic communities',
 'Retail and service industry representatives',
 'Safety and security organizations',
 'Scientific research and researchers',
 'Service recipients and providers',
 'Sewage system users and operators',
 'Social and cultural organizations',
 'Social equity advocates',
 'Social media and communication stakeholder networks',
 "Society's marginalized and vulnerable groups",
 'Specialized sector members',
 'Technological innovators and professionals',
 'Technology and data industry professionals',
 'Technology stakeholders',
 'Tropical climate-affected individuals',
 'Untrained personnel',
 'Urban development and planning entities',
 'Urban planning and architecture groups',
 'Urban planning authorities',
 'Various governmental and non-governmental groups',
 'Vulnerable and marginalized populations',
 'Wastewater and sanitation stakeholders',
 'Workers and employees',
 'Workplace safety advocates']

In [31]:
classifSh = [
        {
        "name": "classify_risks",
        "description": "Function used to classify risks.",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_stakeholder": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of stakeholder ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The stakeholder group being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":LS,
                                "description": 'The category of stakeholders that matches the stakeholder being considered.'
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct stakeholder category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_stakeholder"],
        },
    }
]

In [34]:
RES = []
NChunks = 50

NB = len(SH)//NChunks
CATS = []
for k in range(NB):
    try:
        QUESTION =  "The stakeholders are:\n\n* "+"\n* ".join(SH[(k*NChunks):(k+1)*NChunks]) 

        #answer = ask("Out the following list, classify the risks  \n\n",Rs,prefix="risk_classification",functions=classifFct,overwrite=True)
        prefix ="ff"
        overwrite = False
        REQ = {
            "context": "Out the following list, classify the stakeholders\n\n",
            "question": QUESTION,
            "functions": classifSh,
            "token": TOK3N,
            "overwrite": overwrite,
            "source": "local-evalrisksT3.5"
        }

        H = h.hashme(REQ["context"]+REQ["question"])
        cached = "cache/"+prefix+"_"+H+".json"
        if not os.path.isfile(cached) or overwrite:
            x = requests.post(URL+"fct/", json = REQ)
            answer = json.loads(x.text)["messages"][-1]
            h.svt(cached,answer)
        else:
            answer = h.ldt(cached)
        CATS.append(pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_stakeholder"]))
    except:
        pass

In [37]:
stakeholders = pd.concat(CATS)[["name","category","confidence"]]
stakeholders.to_parquet("newstakeholders_classified.parquet.gzip")